Import dependencies

In [1]:
import numpy as np
import os
import pandas as pd

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

Directory

In [2]:
os.chdir(os.getcwd()+'/images')
print(os.getcwd())

C:\Users\billk\Desktop\Git\spirochete_vision\images


In [3]:
pd.read_csv('data_aug.csv', header=None)

,0,1,2,3,4,5,6,7,8
0,TRAINING,1.jpg,1,0.705208,0.388542,NaN,NaN,0.792708,0.417708
1,VALIDATION,10.jpg,1,0.520833,0.487500,NaN,NaN,0.576042,0.559375
2,TRAINING,100.jpg,1,0.609682,0.630862,NaN,NaN,0.724660,0.717095
3,TRAINING,100.jpg,1,0.667171,0.754917,NaN,NaN,0.829047,0.851740
4,TRAINING,100.jpg,1,0.009077,0.851740,NaN,NaN,0.128593,0.897126
...,...,...,...,...,...,...,...,...,...
985,TRAINING,a7clock.jpg,1,0.045455,0.435065,NaN,NaN,0.220779,0.551948
986,TRAINING,a7clock.jpg,1,0.428571,0.305195,NaN,NaN,0.603896,0.435065
987,TRAINING,a7clock.jpg,1,0.753247,0.097403,NaN,NaN,0.980519,0.240260
988,TRAINING,a8clock.jpg,1,0.340541,0.189189,NaN,NaN,0.535135,0.545946


Specify model

In [4]:
spec = model_spec.get('efficientdet_lite2')

Use data loader

In [5]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('data_aug.csv')

Train model

In [6]:
model = object_detector.create(train_data, model_spec=spec, batch_size=10, epochs=100, train_whole_model=True, validation_data=validation_data)

Epoch 1/100
36/36 [==============================] - 61s 685ms/step - det_loss: 1.6170 - cls_loss: 0.9952 - box_loss: 0.0124 - reg_l2_loss: 0.0759 - loss: 1.6929 - learning_rate: 0.0102 - gradient_norm: 2.7108 - val_det_loss: 1.4163 - val_cls_loss: 0.8459 - val_box_loss: 0.0114 - val_reg_l2_loss: 0.0759 - val_loss: 1.4922
Epoch 2/100
36/36 [==============================] - 20s 558ms/step - det_loss: 0.8626 - cls_loss: 0.4529 - box_loss: 0.0082 - reg_l2_loss: 0.0760 - loss: 0.9386 - learning_rate: 0.0125 - gradient_norm: 4.6375 - val_det_loss: 1.9100 - val_cls_loss: 1.1581 - val_box_loss: 0.0150 - val_reg_l2_loss: 0.0761 - val_loss: 1.9861
Epoch 3/100
36/36 [==============================] - 20s 558ms/step - det_loss: 0.6910 - cls_loss: 0.3674 - box_loss: 0.0065 - reg_l2_loss: 0.0762 - loss: 0.7672 - learning_rate: 0.0125 - gradient_norm: 4.3027 - val_det_loss: 2.0406 - val_cls_loss: 1.2067 - val_box_loss: 0.0167 - val_reg_l2_loss: 0.0763 - val_loss: 2.1169
Epoch 4/100
36/36 [=========

Evaluate

In [7]:
model.evaluate(test_data)

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[39,96,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node keras_layer/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/efficientnet-lite2/StatefulPartitionedCall/blocks_1/tpu_batch_normalization/FusedBatchNormV3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[GatherNd_73/_336]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[39,96,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node keras_layer/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/eff

In [8]:
model.export(export_dir='.')

In [9]:
model.evaluate_tflite('model.tflite', test_data)

39/39 [==============================] - 788s 20s/step



{'AP': 0.5507642,
 'AP50': 0.8582592,
 'AP75': 0.6466581,
 'APs': 0.18445483,
 'APm': 0.58804893,
 'APl': 0.69655716,
 'ARmax1': 0.28645834,
 'ARmax10': 0.61041665,
 'ARmax100': 0.62083334,
 'ARs': 0.29166666,
 'ARm': 0.65833336,
 'ARl': 0.725,
 'AP_/1': 0.5507642}

##### 